In [1]:
from watermark_model import Watermark
import torch
import numpy as np
import argparse
from scipy.stats import normaltest
from watermark_model import *
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.set_printoptions(sci_mode=False,profile='full')

if __name__ == '__main__':
    parser = argparse.ArgumentParser(description='decoder-encoder pretrain')
    parser.add_argument('--secret_length', default=48, type=int)
    args =parser.parse_known_args()[0]
    
model=Watermark(secret_length=args.secret_length).to(device)
model.load_state_dict(torch.load('./model48bit.pth'))
model.eval()

Metric=[]
Loss=[]
for i in range(100):
    secret=torch.Tensor(np.random.choice([0, 1], size=(args.secret_length))).to(device)
    secret = secret.unsqueeze(-1).unsqueeze(-1).unsqueeze(0)
    secret = secret.expand(-1,-1,64,64)
    y, mean, logvar = model(secret)

    mean=mean.reshape(-1,64,64)
    logvar=logvar.reshape(-1,64,64)
    eps = torch.randn_like(logvar)
    std = torch.exp(logvar / 2)
    matrix = eps * std + mean
    average_tensor1 = torch.mean(secret, dim=(-2, -1))
    average_tensor2 = torch.round(torch.mean(y[0], dim=(-2, -1)))
    average_tensor3 = torch.mean(y[0], dim=(-2, -1))
    # print(average_tensor1, average_tensor2, average_tensor3)

    biterror=torch.sum(abs(average_tensor1-average_tensor2))
    mse_loss = torch.nn.MSELoss()
    loss = mse_loss(average_tensor3, average_tensor1)
    Metric.append(biterror)
    Loss.append(loss)
    stat, p1 = normaltest(matrix.view(-1).cpu().detach().clone().numpy())
    print(p1)

biterror=torch.mean(torch.stack(Metric))
biterrper=biterror/args.secret_length
metric=torch.mean(torch.stack(Loss))
print(f'loss bits',f'{biterror.cpu().detach().numpy()}')
print(f'bit error percentage {biterrper*100:.2f}%')
print(f'loss',f'{metric.cpu().detach()}')

Watermark(
  (encoder): Sequential(
    (0): Sequential(
      (0): Conv2d(48, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (1): Sequential(
      (0): Conv2d(48, 96, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (2): Sequential(
      (0): Conv2d(96, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (3): Sequential(
      (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
  )
  (mean_linear): Linear(in_features=4096, out_features=16384, bias=True)
  (var_linear): Linear(in_feat

/root/miniconda3/envs/watermark/lib/python3.10/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([1, 48])) that is different to the input size (torch.Size([48])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


0.05916397328937795
0.8265329188153705
0.04405162668206862
0.4281983809664529
0.4716025675058122
0.7823950190662317
2.5040076104692027e-11
1.0068972267363562e-33
0.3511754879603235
0.07432153181760091
0.6318277245746118
1.5834132142344896e-08
0.22055770978882233
0.005459867704523408
0.000797580834318086
0.0009803694565957107
0.9336332036067448
8.878913517678173e-05
0.16968610603022208
0.6059388878428227
0.0001556939710453973
0.00269949386779281
1.5129251047263362e-06
0.15287790114130148
0.002323798257500189
0.09269571692805763
0.007559008894085567
0.017650726622977914
0.0021175978364794843
0.44273075252648475
9.134560988890323e-06
0.7814260005703942
0.9696615262882043
0.2120368993273042
0.7575831328860702
0.002470546457119504
0.6042928182511855
0.3862004907534134
0.00023758566443306172
0.7855586885367098
0.8298882281913409
0.3369860856729453
1.2483037282453349e-16
0.12853039287849943
0.5119778925172456
0.005779986539691766
1.3763520442133834e-18
0.6759974112044042
0.8637968003708256
3.